# Usecase 2: Temperature prediction original model set-up

This notebook trains an ElasticNet model in Python with the same train-test splits as the ritme runs and a modelling set-up similar to the one described in [the original publication by Sunagawa et al. 2015](https://www.science.org/doi/10.1126/science.1261359).


This notebook can be run in the following conda environment:

```shell
mamba create -n ritme_model -c adamova -c qiime2 -c conda-forge -c bioconda -c pytorch -c anaconda ritme ipykernel -y
conda activate ritme_model
pip install -e .
```


Description of modelling set-up used by original publication:           
"Compositional data (see above) were normalized to ranks across samples and then used to learn a regression model to predict environmental measures. In particular, we fitted an elastic net model (44) using inner cross-validation to set the hyperparameters as implemented by the scikit-learn Python package"

For a proper comparison with ritme, we are running inner cross-validation on the train set and evaluating the resulting model on the test set.

## Setup

In [ ]:
import pandas as pd
from sklearn.linear_model import ElasticNetCV

from src.eval_originals import get_metrics_n_scatterplot

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
######## USER INPUTS ########

# path to folder where train-test splits used for ritme are stored
data_splits_folder = "data_splits_u2"

# path to processed feature table used by original publication
path_to_suna_ft = "../../data/u2_tara_ocean/otu_table_tara_ocean_proc.tsv"
path_to_suna_md = "../../data/u2_tara_ocean/md_tara_ocean.tsv"

######## END USER INPUTS #####

## Prepare data

In [ ]:
# Get indices of training and test data used by ritme
ritme_train_df = pd.read_pickle(f"{data_splits_folder}/train_val.pkl")
ritme_test_df = pd.read_pickle(f"{data_splits_folder}/test.pkl")

train_idx = ritme_train_df.index.tolist()
test_idx = ritme_test_df.index.tolist()

In [ ]:
# select these indices from sunagawa md + feature table
suna_ft = pd.read_csv(path_to_suna_ft, sep="\t", index_col=0)
predictor_cols = suna_ft.columns

suna_md = pd.read_csv(path_to_suna_md, sep="\t", index_col=0)

suna_data = suna_md.join(suna_ft, how="inner")

In [ ]:
train_df = suna_data.loc[train_idx]
test_df = suna_data.loc[test_idx]

print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

assert train_df.shape[0] == len(train_idx)
assert test_df.shape[0] == len(test_idx)

In [ ]:
# Extract predictors and target
train_predictors = train_df[predictor_cols]
train_target = train_df["temperature_mean_degc"]

test_predictors = test_df[predictor_cols]
test_target = test_df["temperature_mean_degc"]

## Train & evaluate ElasticNet model in Python

In [ ]:
# Train ElasticNet with inner 5-fold CV on ranked features
enet_cv = ElasticNetCV(
    l1_ratio=[0.1, 0.5, 0.9, 1.0],
    cv=5,
    random_state=123,
    n_jobs=-1,
)
enet_cv.fit(train_predictors, train_target)

In [ ]:
# evaluate
metrics, fig = get_metrics_n_scatterplot(
    enet_cv, train_predictors, train_target, test_predictors, test_target
)
display(metrics)